<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/121_%E3%83%87%E3%83%BC%E3%82%BF%E6%A7%8B%E9%80%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

データ構造
==========

リストと配列
------------

### リストの基本 ###

リストについては、公式チュートリアルは 2 つのセクションに分けて解説していた。

  * [リスト型 (list)](https://docs.python.org/ja/3/tutorial/introduction.html#lists)
  * [リスト型についてもう少し](https://docs.python.org/ja/3/tutorial/datastructures.html#more-on-lists)

また、シーケンスの比較については、公式チュートリアルは次のセクションで解説していた。

  * [シーケンスとその他の型の比較](https://docs.python.org/ja/3/tutorial/datastructures.html#comparing-sequences-and-other-types)

リストはいくつかの方法で生成できる:

  * 角括弧の対を使い、空のリストを表す: `[]`
  * 角括弧を使い、要素をカンマで区切る: `[a]`、`[a, b, c]`
  * リスト内包表記を使う: `[x for x in iterable]`
  * 型コンストラクタを使う: `list()` または `list(iterable)`

指定したサイズの初期化されたリストは、`*` 演算子による反復で生成するのが最速である。

In [ ]:
[0] * 5

[0, 0, 0, 0, 0]

リストを空にするには、大きく 2 つの方法がある。

  1. リストの `clear()` メソッドを呼び出す。 `del a[:]` と等価であるが、 `clear()` メソッドの方が効率的で、可読性が高い。
  2. 新たな空リストを代入する。

1 の方法では同じリストを参照している他の変数にも影響するのに対して、 2 の方法ではもとのリストを参照している他の変数には影響しない。また、 1 の方法では新たな空リストを作成するコストがかからない。他の変数が同じリストを参照している可能性がある場合や、パフォーマンスが重要な場合は 1 の方法を使用するとよい。

In [ ]:
a = [x for x in range(5)]
b = a
a.clear()  # del a[:] と等価
assert b == []  # 変数 b にも影響する

a = [x for x in range(5)]
b = a
a = []
assert b == [0, 1, 2, 3, 4]  # b はもとのリストを参照している

### array ###

標準ライブラリの `array` モジュールは、要素の型に制限があるリスト `array.array` をサポートする。コンストラクタの引数は以下の通り。

``` python
array.array(typecode[, initializer])
```
`typecode` 引数には、Python の型ではなく、実装上の言語がサポートする型を直接指定する。 CPython では、以下の型を指定できる。

| `typecode` | C の型 | バイト |
|:--:|:---|:--:|
| `'b'` | signed char | 1 |
| `'B'` | unsigned char | 1 |
| `'u'` | wchar_t | 2 |
| `'i'` | signed int | 2 |
| `'I'` | unsigned int | 2 |
| `'l'` | signed long | 4 |
| `'L'` | unsigned long | 4 |
| `'q'` | signed long long | 8 |
| `'Q'` | unsigned long long | 8 |
| `'f'` | float | 4 |
| `'d'` | double | 8 |

Python のデータ型は種類が少なく、 CPython では `int` は多倍長整数、`float` は 64 ビットの倍精度浮動小数点数となる。 `array.array` クラスのインスタンスなら、メモリ節約のために小さなサイズのデータ型を扱うことができる。

In [ ]:
import array, sys
assert bytes(array.array('B', [0x41])) == b'A'
assert bytes(array.array('B', [0xe3, 0x81, 0x82])) == b'\xe3\x81\x82' == bytes('あ', 'utf-8')
lst = [i for i in range(10000)]
arr = array.array('i', lst)
print("List size:  ", sys.getsizeof(lst))
print("Array size: ", sys.getsizeof(arr))

List size:   85176
Array size:  40080


`array.array` は、通常のリストとほとんど同じように使用することができるが、`clear()`, `copy()`, `sort()` の 3 つのメソッドが実装されていないことに注意する。よほどメモリ制約が厳しい環境でもない限り、`array.array` をあえて使用するメリットはないと思う。

### リスト操作の計算量 ###

誤解されやすいことであるが、CPython のリストの実装は、Lisp にみられる連結リストではなく、**動的配列**（＝要素の追加・削除が可能な配列）である。この実装は、他のオブジェクトへの参照を要素とする配列を使用していて、この配列への参照と配列長がリストの先頭の構造体に保存されるというものである。

これにより、リストのインデックス参照 `a[i]` は、リストのサイズやインデクスの値に依存しない計算量、つまり $O(1)$ で計算される（公式 Wiki のページ [TimeComplexity](https://wiki.python.org/moin/TimeComplexity) 参照）。

一方、`index()` メソッドである値の位置を探索する場合、愚直に先頭からしらみつぶしで調べる方法（線形探索）となり、目的の値が末尾にある場合が最悪で $O(n)$ と低速である（$n$ はリストのサイズ）。

また、リストの実装が配列であることから、末尾での要素の挿入や削除（`append()` や `pop()` 操作）は計算量が $O(1)$ と高速だが、末尾以外の位置での要素の挿入や削除（`insert(i, x)` や `del a[i]` 操作）は $O(n)$ と低速である（公式 Wiki のページ [TimeComplexity](https://wiki.python.org/moin/TimeComplexity) 参照）。

末尾以外の位置での要素の挿入や削除が低速となるのは、既存の要素の移動が必要となるからである。リスト `a` の `i` 番目の位置に要素 `x` を挿入する場合（`a.insert(i, x)`）、まずリストのサイズを増やして `i+1` 番目以降の要素を 1 つずつ右に移動させる必要がある。

``` text
    0     1             i     i+1   i+2           n-1   n
   ┌──┬──┬      ┬──┬──┬──┬      ┬──┬──┐
a  │    │    │  …  │    │    │    │  …  │    │    │
   └──┴──┴      ┴──┴──┴──┴      ┴──┴──┘
      ↓    ↓           移動   ↓    ↓            ↓    ↓
     a[0]  a[1]           ⇒   a[i] a[i+1]        a[n-1] a[n]
```

とくに先頭位置での挿入（`a.insert(0, x)`）が最悪で、`n` 個の要素をそれぞれ右に移動させる必要がある（移動という処理が `n` 回必要となる）。

リスト `a` の `i` 番目の位置にある要素を削除する場合（`del a[i]`）、`i+1` 番目以降の要素を 1 つずつ左に移動させ、リストのサイズを減らすことになる。とくに先頭位置での削除（`del a[0]`）が最悪で、`n-1` 回の移動が必要である。

一方、末尾での要素の挿入や削除（`append()` や `pop()` 操作）では、既存の要素を移動する必要はない。`append()` メソッドでは末尾に要素を追加してリストのサイズを増やすだけで、既存の要素はそのままでよい。 `pop()` メソッドでは末尾の要素を取り出してリストのサイズを減らすだけで、既存の要素はそのままでよい。そのため、高速である。

二分探索・ヒープ・キュー・スタック
----------------------------------

要素の探索や、先頭での挿入・削除が低速であるというリストの弱点を補うためのアルゴリズムやデータ構造が考案されている。

### 配列二分法 ###

**二分探索**（binary search）は、ソート済みリストに対する探索アルゴリズムの 1 つであり、リストの中間点を求めて値が中間点より小さければ左側半分の部分リストを取り、大きければ右側半分の部分リストを取るという操作を再帰的に行うものである。計算量は $O(\log_{2}n)$ である。

Python は、二分探索そのものを行う関数やモジュールを提供していない。その代わりに、標準ライブラリの `bisect` モジュールが、二分探索を使用して値の挿入位置を見つける関数を提供している。なお、モジュール名 `bisect` の由来は**二分法**（bisection algorithm）らしいが、一般には二分法は方程式を解く求根アルゴリズムを指すので注意。

``` python
bisect.bisect_left(a, x, lo=0, hi=len(a), *, key=None)
bisect.bisect_right(a, x, lo=0, hi=len(a), *, key=None)
bisect.bisect(a, x, lo=0, hi=len(a), *, key=None)
```

これらの関数は、ソート済みのリスト `a` に、ソートされた順序を保ったままオブジェクト `x` を挿入できる位置（インデックス値）を返す。`x` が `a` に含まれている場合は、`bisect.bisect_left()` は最初の `x` の位置を返すのに対して、`bisect.bisect_right()` は最後の `x` の後ろの位置を返す。`bisect.bisect()` は、`bisect.bisect_right()` の別名である。その他の共通する引数については、次のとおり。

| 引数 | 意味 |
|:---|:---|
| `lo` | 検索開始位置を指定する。省略時は先頭から検索する |
| `hi` | 検索終了位置を指定する。省略時は末尾まで検索する |
| `key` | キーワード専用引数 `key` は、引数を 1 つ受け取る呼び出し可能オブジェクトであり、配列内の各要素に何らかの処理を行った結果をもとに大小比較を行う。<br />`None` の場合は、要素は直接<br />比較される |

応用例:

  * `a` の中の `x` を含む部分リストは `a[bisect.bisect_left(a, x) : bisect.bisect_right(a, x)]` で取得できる。
  * `bisect.bisect_left(a, x) < len(a)` と `x in a` は等価である。
  * `bisect.bisect_left(a, x) == len(a)` と `x not in a` は等価である。

`in`, `not in` や リストの `index()` メソッドでは、要素をリストの前から順番に調べる線形探索を行うため、要素がリストの後ろのほうにある場合には処理時間が長くなる。以下のコードで `bisect` のパフォーマンスを検証する。

In [ ]:
import bisect
a = list(range(100_000_000))
x = 90_000_000
%timeit bisect.bisect_left(a, x)
%timeit x in a

1.12 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.17 s ± 189 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ソート済みリストに要素が含まれるなら最初のほうにあることがわかっている場合や、リストが十分にランダムな場合には、`in`, `not in` や `index()` メソッドを使ったほうが速い。

`bisect` モジュールは、ソート済みのリストにソートされた順序を保ったままオブジェクトを挿入する関数も用意している。

``` python
bisect.insort_left(a, x, lo=0, hi=len(a), *, key=None)
bisect.insort_right(a, x, lo=0, hi=len(a), *, key=None)
bisect.insort(a, x, lo=0, hi=len(a), *, key=None)
```

`bisect.insort*(a, x)` は `i = bisect.bisect*(a, x); a.insert(i, x)` と等価である。

### 優先度付きキュー ###

二分探索は、対象とするリストがソート済みである必要がある。 CPython でのソートの実装は、計算量が $O(n \log_{2} n)$ となるアルゴリズムが使用されている。

常に最小値を探索するのであれば、リスト全体のソートを必要としないアルゴリズムがある。リスト内の要素の位置を、常に**ヒープ**（heap）と呼ばれる木構造に対応するようにしている場合

  * 最小値を $O(\log_{2} n)$ 時間で取り出す
  * 要素を $O(\log_{2} n)$ 時間で挿入する

という計算量を実現する。ヒープは、「子要素は親要素より常に大きいか等しい」という制約を持つ二分木である。この制約を**ヒープ不変式**と呼ぶ。リストの要素がヒープに対応する順序で並んでいることを、「ヒープ不変式を保つ」と表現する。

たとえば、下の図の木はヒープである（画像は [File:Binary heap indexing.png](https://commons.wikimedia.org/wiki/File:Binary_heap_indexing.png) より）。

![](https://upload.wikimedia.org/wikipedia/commons/6/60/Binary_heap_indexing.png)

この図の場合、ヒープ不変式を保つリストは、根の要素から始まり上から順に左から要素を並べた形

$$
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
$$

である。

ヒープ不変式を保つリストに対して要素の挿入・削除を行う場合、ヒープ不変式が保たれるように要素の並べ替えが行われる。空でないリストに対してヒープ不変式が保たれるように最初に行う要素の並べ替えを**ヒープ化**（heapify）という。ヒープ化のアルゴリズムは、計算量が $O(n)$ となる。一度ヒープ化すれば、それ以降、ヒープ不変式を保つ挿入・削除の計算量は、$O(\log n)$ で済む。なぜなら、一からヒープを作り直すのではなく、ヒープ不変式を満たしていない親子関係だけ要素を入れ替えればよいからである。

ヒープ不変式から、当然に根の要素が最小値である。これはリストの先頭の要素が最小値であることを意味する。最小値を取り出すとき、次の要素が残りの要素のうちの最小値であるとは限らないが、削除時の要素入れ替え作業により先頭が再び最小値になる。

ヒープ不変式を保つリストは、値を小さい要素から順に取り出すことのできるデータ構造と見ることができる。このようなデータ構造は**優先度付きキュー**（priority queue）と呼ばれる。この場合の優先度は「値が小さい要素ほど優先度が高い」というものである。

標準ライブラリの `heapq` モジュールは、リストを、ヒープ不変式を保つように操作するための関数を提供している。

``` python
heapq.heappush(heap, item)
```

この関数は、`item` を `heap` に挿入する。ヒープ不変式を保つ。`heap` は、ヒープ化されたリストまたは空リストでなければならない。

``` python
heapq.heappop(heap)
```

この関数は、`heap` から最小の要素を取り出す。ヒープ不変式を保つ。`heap` は、ヒープ化されたリストでなければならない。`heap` が空の場合、 `IndexError` 例外が送出される。

``` python
heapq.heapify(x)
```

この関数は、リスト `x` をヒープ化する。

`heapq` モジュールを使用すると、 `heapq.heappop()` で最小値を取得できる。 `heapq` モジュールの関数を使わずに毎回組み込み関数 `min()` を使う場合、 `min()` は線形探索を行うので計算量が $O(n)$ となり、 `heapq` モジュールの関数を使う場合よりかなり効率が悪くなる。

次のコードは、 `heapq` モジュールの使用例である。

In [ ]:
import heapq

# 既存のリストをヒープ化
data = [90, 34, 78, 65, 56]
heapq.heapify(data)
assert data[0] == min(data)
print(f"ヒープ化後: {data=}")

# 要素の追加
heapq.heappush(data, 12)
heapq.heappush(data, 28)
assert data[0] == min(data)
print(f"heappush後: {data=}")

# 最小要素の取り出し
assert min(data) == heapq.heappop(data)
assert data[0] == min(data)
print(f"heappop後:  {data=}")

ヒープ化後: data=[34, 56, 78, 65, 90]
heappush後: data=[12, 56, 28, 65, 90, 78, 34]
heappop後:  data=[28, 56, 34, 65, 90, 78]


得られた出力から、`heapq` の関数によるリストの並べ替えは `list.sort()` とは異なること、および、先頭の要素が常に最小値であることがわかる。

### deque ###

先頭位置での挿入や削除が苦手な Python のリストは、キュー、スタック、デック（両端キュー）のようなデータ構造には不向きである。

**キュー**（queue）あるいは**待ち行列**は、先に入れられたデータから順に取り出される、いわゆる「先入れ先出し」（First In, First Out; FIFO）というデータ構造である。キューにデータを入れることを**エンキュー**（enqueue）、取り出すことを**デキュー**（dequeue）という。下図（[Wikipedia の記事](https://ja.wikipedia.org/wiki/キュー_(コンピュータ))から引用）はキューの単純な表現である。

<a href="https://commons.wikimedia.org/wiki/File:Data_Queue.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/Data_Queue.svg/320px-Data_Queue.svg.png" width="240"></a>

**スタック**（stack）は、キューとは逆に「後入れ先出し」（Last In, First Out; LIFO）というデータ構造である。スタックにデータを入れることを**プッシュ**（push）、取り出すことを**ポップ**（pop）という。下図（[Wikipedia の記事](https://ja.wikipedia.org/wiki/スタック)から引用）はキューの単純な表現である。

<a href="https://commons.wikimedia.org/wiki/File:Data_Queue.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/93/Stack-sv.svg/320px-Stack-sv.svg.png" width="240"></a>

**デック**（deque）あるいは**両端キュー**は、先頭または末尾で要素を追加・削除できるキューである。deque は double-ended queue の略で、英語の発音は "deck" である（デキューと読むのは誤り）。

標準ライブラリの `collections` モジュールは、デックに特化したシーケンスの C 実装であるクラス `deque` を提供する。コンストラクタは次のとおり。

``` python
collections.deque([iterable[, maxlen]])
```

第 1 引数としてイテラブルが渡された場合、その要素が順番通りに `collections.deque` に入れられる形で初期化される。第 1 引数にイテラブルが与えられない場合、新しい `collections.deque` オブジェクトは空になる。

第 1 引数または第 2 引数として整数が渡された場合、`collections.deque` オブジェクトのサイズは指定された最大長に制限され、サイズが制限いっぱいになると、新しい要素を追加するときに追加した要素数分だけ追加したのと反対側から要素が捨てられる。`collections.deque` オブジェクトのサイズの最大長は、読み出し専用プロパティ `maxlen` で参照できる。

引数が無かったり、 `None` だった場合、`collections.deque` は任意のサイズまで大きくなる。この場合、`maxlen` プロパティの値は常に `None` である。

`collections.deque` は、シーケンスなのでリストと同様に共通のシーケンス演算をサポートしている。すなわち、 `collections.deque` に対して `in`、`not in`、`+`、`*`、`[]`（スライスは使用できないことに注意）が使える。 `len()` でサイズを取得したり、 `min()` で最小値を取得したり、イテラブルとして使用することもできる。

また、 `collections.deque` は、リストと同様のメソッドをサポートしており、さらにいくつかの追加メソッドもサポートする。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `append(x)` | `x` をオブジェクトの右側（末尾）に追加する | `None` |
| `appendleft(x)` | `x` をオブジェクトの左側（先頭）に追加する | `None` |
| `extend(iterable)` | イテラブルな引数 `iterable` から得られる要素をオブジェクトの右側（末尾）に追加する形で拡張する | `None` |
| `extendleft(iterable)` | イテラブルな引数 `iterable` から得られる要素をオブジェクトの左側（先頭）に追加する形で拡張する | `None` |
| `insert(i, x)` | `x` をオブジェクトの位置 `i` に挿入する。オブジェクトがサイズ制限付きの場合、挿入によってサイズが制限を超えるなら<br /> `IndexError` 例外を送出する | `None` |
| `pop()` | オブジェクトの右側（末尾）から要素を 1 つ削除し、その要素を返す。要素が 1 つも存在しない場合は `IndexError` 例外<br />を送出する | 要素 |
| `popleft()` | オブジェクトの左側（先頭）から要素を 1 つ削除し、その要素を返す。要素が 1 つも存在しない場合は `IndexError` 例外<br />を送出する | 要素 |
| `remove(value)` | 値が `value` である要素で最初に現れるものを削除する。要素が見付からないない場合は `ValueError` 例外を送出する | `None` |
| `clear()` | すべての要素を削除し、長さを 0 にする | `None` |
| `rotate(n=1)` | 要素を全体で `n` ステップだけ右に循環させる。`n` が負の値の場合は、左に循環させる | `None` |
| `reverse()` | オブジェクトの要素をインプレースに反転する | `None` |
| `index(x[, start[, stop]])` | オブジェクト内の `x` の位置を返す（位置 `start` から位置 `stop` の両端を含む範囲で）。`x` が見つからない場合には<br /> `ValueError` 例外を送出する | `int` |
| `count(x)` | オブジェクトの `x` に等しい要素を数え上げる | `int` |
| `copy()` | オブジェクトの浅いコピーを作成する | `deque` |

In [ ]:
from collections import deque, abc
assert issubclass(deque, abc.Sequence)  # deque はシーケンスである
d = deque([i for i in range(1, 5)], 6)
print("original:  ", d)
d.append(5)
print("append:    ", d)
d.appendleft(0)
print("appendleft:", d)
d.append(6)
print("append:    ", d, " ←制限超過のため先頭が捨てられる")
d.appendleft(0)
print("appendleft:", d, " ←制限超過のため末尾が捨てられる")
assert d.pop() == 5
print("pop:       ", d)
assert d.popleft() == 0
print("popleft:   ", d)
d.reverse()
print("reverse:   ", d)
d.rotate()
print("rotate:    ", d)
assert len(d) == 4
assert sorted(d) == [1, 2, 3, 4]
assert d[0] == 1

original:   deque([1, 2, 3, 4], maxlen=6)
append:     deque([1, 2, 3, 4, 5], maxlen=6)
appendleft: deque([0, 1, 2, 3, 4, 5], maxlen=6)
append:     deque([1, 2, 3, 4, 5, 6], maxlen=6)  ←制限超過のため先頭が捨てられる
appendleft: deque([0, 1, 2, 3, 4, 5], maxlen=6)  ←制限超過のため末尾が捨てられる
pop:        deque([0, 1, 2, 3, 4], maxlen=6)
popleft:    deque([1, 2, 3, 4], maxlen=6)
reverse:    deque([4, 3, 2, 1], maxlen=6)
rotate:     deque([1, 4, 3, 2], maxlen=6)


`collections.deque` の場合、要素の追加と取り出しが両側ともに高速である。すなわち、 `append()`、`appendleft()`、`pop()`、`popleft()` メソッドは計算量が $O(1)$ であり、計算時間がオブジェクトのサイズ $n$ に影響されない。一方、インデックス参照が両端では $O(1)$ であるが、中央ではサイズ $n$ に線形 $O(n)$ と遅くなる（公式 Wiki のページ [TimeComplexity](https://wiki.python.org/moin/TimeComplexity) 参照）。

リストのインデックス参照は要素の位置に関係なく計算量が $O(1)$ と安定して高速であるから、高速ランダムアクセスが必要な場合は、 `collections.deque` よりリストが適している。

数十万件のような大規模なデータをキューやスタック、デックとして扱うことが目的ならば、`collections.deque` が適している。

`collections.deque` をキューとして使うには、エンキューとして `append()`、デキューとして `popleft()` を使えばよい。

`collections.deque` をスタックとして使うには、プッシュとして `append()`、ポップとして `pop()` を使えばよい。

In [ ]:
items = [i for i in range(500000)]
d = deque(items)
%timeit d.appendleft(0)
%timeit items.insert(0, 0)
%timeit d[250000]
%timeit items[250000]

86.3 ns ± 16.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
271 µs ± 8.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
20.9 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
69.1 ns ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


`collections.deque` 型の型アノテーションは、`list` と同様に、添字表記（`[]`）を使って要素の型を指定しなければならない。

In [ ]:
d: deque[int | None] = deque()
d.append("hoge")  # 静的型チェッカーがエラーを表示する

集合
----

数学では、「集合」は「ものの集まり」であって、以下の性質を持つとされる。

  1. 要素の並び順には意味はない──要素の順序に意味を持つ場合は「列」と呼ばれる
  2. 要素は重複してはいけない──重複を許す集まりは「多重集合」と呼ばれる

組み込み型 `set` と `frozenset` は、このような集合を Python に実装したものであり、ハッシュ値を使って要素を管理するコレクションである。集合の要素は、ハッシュ可能なオブジェクトに限られる。同一の要素は、 `__hash__()` が同じハッシュ値を返すため排除される。

集合は、他のコレクションと同様、 `x in set`、 `len(set)`、 `for x in set` をサポートする。コレクションには順序がないので、集合は挿入の順序や要素の位置を記録しない。したがって、集合はインデックス参照などのシーケンス的な振舞いをサポートしない。

`set` オブジェクトはいくつかの方法で生成できる:

  * 波括弧内にカンマ区切りで要素を列挙する: `{'jack', 'sjoerd'}`
  * 集合内包表記を使う: `{c for c in 'abracadabra' if c not in 'abc'}`
  * 型コンストラクタを使う: `set([iterable])`

`frozenset` には専用のリテラル構文はないので、コンストラクタ `frozenset([iterable])` を使ってオブジェクトを生成する。

`{}` は空の集合ではなく空の辞書を生成することに注意する。空の集合は、引数を渡さない `set()` と `frozenset()` 呼び出しで作成できる。

`set` はミュータブルな集合で、以下のメソッドを呼び出して内容を変更できる。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `add(elem)` | 要素 `elem` を `set` に追加する | `None` |
| `remove(elem)` | 要素 `elem` を `set` から取り除く。`elem` が `set` に含まれていなければ `KeyError` 例外を送出する | `None` |
| `pop()` | `set` から何らかの要素を取り除き、それを返す。集合が空の場合、`KeyError` 例外を送出する | `None` |

`set` オブジェクト自身はハッシュ値を持たず、辞書のキーや他の集合の要素として用いることができない。

一方、`frozenset` はイミュータブルな集合で、作成後に内容を変更できない。また、`frozenset` オブジェクト自身もハッシュ可能であり、辞書のキーや他の集合の要素として用いることができる。

`copy()` メソッドで集合の浅いコピーを返す。

集合同士の比較（`<`, `<=`, `>`, `>=`）が可能で、包含関係で評価される。たとえば、`seta < setb` は `seta` が `setb` の真部分集合である場合に真となる。

集合演算は、リストにはない機能である。 `|` で和集合、 `&` で積集合、 `-` で差集合を表す。集合演算は、常に新しい集合を返す。 `set` オブジェクトと `frozenset` オブジェクトを取り混ぜての二項演算は、左側の型を返す。

これら集合演算の結果で左側の `set` オブジェクトの内容を更新するには、短縮形式 `|=`、 `&=`、 `-=` を使うことができる。

In [ ]:
# set と frozenset の比較が可能
assert set(["foo", "baa"]) == frozenset(["foo", "baa"])
assert set(["foo", "baa"]) < frozenset(["foo", "baa", "boo"])

hoge = set("hoge")
fuga = frozenset("fuga")
print(f"{hoge = }")
print(f"{fuga = }")
print(f"{hoge | fuga = }")
print(f"{fuga & hoge = }")
print(f"{hoge - fuga = }")
hoge2 = hoge
hoge -= fuga  # hoge = hoge - fuga と同等
print(f"{hoge = }")
print(f"{hoge2 = }")  # hoge2 が参照するミュータブルなオブジェクト自体が変更されている

hoge = {'h', 'e', 'o', 'g'}
fuga = frozenset({'f', 'u', 'a', 'g'})
hoge | fuga = {'h', 'g', 'o', 'f', 'u', 'a', 'e'}
fuga & hoge = frozenset({'g'})
hoge - fuga = {'h', 'e', 'o'}
hoge = {'h', 'e', 'o'}
hoge2 = {'h', 'e', 'o'}


辞書
----

### 辞書の生成 ###

辞書はいくつかの方法で生成できる:

  * 波括弧内にカンマ区切りで `key: value` 対を列挙する: `{'jack': 4098, 'sjoerd': 4127}`
  * 辞書内包表記を使う: `{}`, `{x: x ** 2 for x in range(10)}`
  * 型コンストラクタを使う: `dict(**kwargs)`, `dict(mapping, **kwargs)`, `dict(iterable, **kwargs)`
  * クラスメソッドを使う: `dict.fromkeys(iterable[, value])`

`dict.fromkeys()` はクラスメソッドであり、`iterable` からキーを取り、値を `value` で初期化された新しい辞書を作成する。`value` はデフォルトで `None` となる。

In [ ]:
assert dict() == {}  # 空の辞書
a = dict(one=1, two=2, three=3)
b = dict({'one': 1, 'two': 2, 'three': 3})
c = dict({'one': 1, 'two': 2}, three=3)
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
f = {'one': 1, 'two': 2, 'three': 3}
assert a == b == c == d == e == f
assert dict.fromkeys(range(3)) == {x: None for x in range(3)} == {0: None, 1: None, 2: None}
assert dict.fromkeys('ABC', 0) == {x: 0 for x in 'ABC'} == {'A': 0, 'B': 0, 'C': 0}

`dict.fromkeys()` で作られる辞書では、すべての値が同一のインスタンスを指すことになるため、`value` にミュータブルなオブジェクト（たとえば空のリスト）を指定しても通常意味はない。別々の値を指すようにしたい場合は、代わりに辞書内包表記を使用すべきである。

In [ ]:
d1 = dict.fromkeys('ABC', [])
assert d1 == {'A': [], 'B': [], 'C': []}  # 各値は同一の空リストを参照している
d1["A"].append(0)
assert d1 == {'A': [0], 'B': [0], 'C': [0]}
d2 = {x: [] for x in 'ABC'}
assert d2 == {'A': [], 'B': [], 'C': []}  # 各値は別の空リストを参照している
d2["A"].append(0)
assert d2 == {'A': [0], 'B': [], 'C': []}

辞書にキーを追加した順序は記憶される。キーに対する値を更新しても、この順序は変更されない。また、辞書は組み込み関数 `reversed()` による逆順の反復もサポートしている。

二つの辞書について比較 `==` は、同じ `(key, value)` の対を持つ場合に、そしてその場合にのみ真と評価され、挿入順序には影響されない。

In [ ]:
d1 = {k: v for k, v in enumerate("AbC")}
print(f"{d1=}")
d1[1] = "B"  # 値を更新しても順序は変化しない
print(f"{d1=}")
d2 = {k: d1[k] for k in reversed(d1)}  # キーを逆順で取得
print(f"{d2=}")
assert d1 == d2  # 辞書の等価判定は、順序に影響されない

d1={0: 'A', 1: 'b', 2: 'C'}
d1={0: 'A', 1: 'B', 2: 'C'}
d2={2: 'C', 1: 'B', 0: 'A'}


リストの重複する要素を削除するには、集合型に変換してからリストに戻すという方法があるが、この方法では要素の出現順が保持されない。リストの要素をキーとする辞書に変換してからリストに戻すという方法を使うと、要素の出現順を保持したまま、重複する要素を削除することができる。

In [ ]:
li1 = list("ABCABC")
li2 = list(set(li1))  # いったん集合型に変換することで重複要素を削除する
print(f"{li2=}")  # 要素の出現順は保持されない
li3 = list(dict.fromkeys(li1))
print(f"{li3=}")  # 要素の出現順は保持される

li2=['C', 'A', 'B']
li3=['A', 'B', 'C']


### 辞書の操作 ###

キーの探索:

| 演算子 | 機能 | 戻り値 |
|:---|:---|:---|
| `key in d` | `d` がキー `key` を持っていれば `True` を、そうでなければ、 `False` を返す | `bool` |
| `key not in d` | `not key in d` と等価 | `bool` |

値の取得:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `get(key[, default])` | `key` が辞書にあれば `key` に対する値を、そうでなければ `default` を返す。`default` が与えられなかった場合、デフォ<br />ルトでは `None` を返す。このメソッドは `KeyError` 例外を送出することはない | 値 |
| `setdefault(key[, default])` | `key` が辞書にあれば `key` に対する値を返し、そうでなければ値を `default` として `key` を挿入し、`default` を返す。<br />`default` のデフォルトは `None` | 値 |
| `pop(key[, default])` | `key` があればその値を辞書から消去して返し、そうでなければ `default` を返す。`default` が与えられず、かつ `key` が<br />辞書に存在しなければ `KeyError` 例外を送出する | 値 |
| `popitem()` | 後入れ先出し（LIFO）の順序で `(key, value)` 対を辞書から消去して返す。辞書が空であれば、`popitem()` の呼び出し<br />は `KeyError` 例外を送出する | `tuple` |

辞書の変更その他:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `update([other])` | 辞書の内容を `other` のキーと値で更新する。既存のキーは上書きされる。`update()` は、他の辞書オブジェクトでもキー/値の<br />対のイテラブル（タプル、もしくは、長さが2のイテラブル）でも、どちらでも受け付ける。キーワード引数が指定されれば、その<br />キー/値の対で辞書を更新する: `d.update(red=1, blue=2)` | `None` |
| `clear()` | 辞書を空にする | `None` |
| `copy()` | 辞書の浅いコピーを返す | `dict` |
| `keys()` | 辞書のキーの新しいビューを返す | 辞書ビュー |
| `values()` | 辞書の値の新しいビューを返す | 辞書ビュー |
| `items()` | 辞書の項目（`(key, value)` 対）の新しいビューを返す | 辞書ビュー |

In [ ]:
people = {'John': 1, 'Emma': 2, 'Sophia': 3, 'James': 4}
assert 'Sophia' in people
assert 'Eric' not in people
assert people.get('Eric') is None  # people['Eric'] では KeyError が発生する
assert people.setdefault('Eric', 0) == 0
print(people)
assert people.popitem() == ('Eric', 0)
print(people)
assert people.pop('John') == 1
print(people)
people.update({'John':10, 'Emma': 20})
people.update([('Sophia', 30), ('Eric', 50)])
people.update(James=40, Oliver=60)
print(people)

{'John': 1, 'Emma': 2, 'Sophia': 3, 'James': 4, 'Eric': 0}
{'John': 1, 'Emma': 2, 'Sophia': 3, 'James': 4}
{'Emma': 2, 'Sophia': 3, 'James': 4}
{'Emma': 20, 'Sophia': 30, 'James': 40, 'John': 10, 'Eric': 50, 'Oliver': 60}


特殊メソッド:

| メソッド | 機能 |
|:---|:---|
| `__missing__(key)` | `self[key]` の実装において辞書内にキーが存在しなかった場合に、`dict` のサブクラスのために `dict.__getitem__()` によって呼び<br />出される |

In [ ]:
class CustomDict(dict):
    def __missing__(self, key):
        return str(key)

d = CustomDict()
assert d['a'] == 'a'  # 'a' キーは存在しないが、__missing__('a') の戻り値が返され、KeyError は発生しない
d

{}

文 `del d[key]` で辞書 `d` から `d[key]` を削除する。辞書に `key` が存在しなければ、`KeyError` 例外が発生する。

`d | other` は、辞書 `d` と `other` のキーと値を統合した新しい辞書を作成する。`d` と `other` のキーに重複がある場合は、`other` の方の値が優先される（Python 3.9 で追加）。

`d |= other` は、辞書 `d` のキーと値を `other` で更新する。`other` はマッピングか、またはキーと値のペアのイテラブルである。`d` と `other` のキーに重複がある場合は、`other` の方の値が優先される（Python 3.9 で追加）。

In [ ]:
assert {1:'あ', 2: 'い'} | {2:'う'} == {1: 'あ', 2: 'う'}
d1 = {1: 'A'}
d1 |= [(2, 'B'), (3, 'C')]
assert d1 == {1: 'A', 2: 'B', 3: 'C'}  # d1.update([(2, 'B'), (3, 'C')]) と等価

浅いコピーと深いコピー
----------------------

### 浅いコピー ###

Python では、変数の代入はオブジェクトのコピーではなくオブジェクトの参照、つまりメモリ上のアドレスが格納される。これは、オブジェクトがイミュータブル（変更不可）でもミュータブル（変更可能）でも同じであるが、とくにミュータブルの場合、その値の変更が、オブジェクトを参照する全ての変数に影響する結果をもたらす。

In [ ]:
vals = ["a", "b", "c", "d"]
v_ref = vals  # オブジェクトの値ではなく参照が代入される
assert v_ref == ["a", "b", "c", "d"]
v_ref[1] = "e"
assert vals == v_ref == ["a", "e", "c", "d"]  # 変更が vals にも及んでいる

この結果を回避するには、オブジェクトのコピーを変数に代入する。

以下のオブジェクトの `copy()` メソッドは、コピーを返す。

  * リストオブジェクト
  * `collections.deque` オブジェクト
  * 集合オブジェクト
  * 辞書オブジェクト

また、組み込み関数を以下のように呼び出す方法でもコピーが得られる。

  * `list(リストオブジェクト)`
  * `set(集合オブジェクト)`
  * `dict(辞書オブジェクト)`

リストオブジェクトについてはスライスを使用してもコピーが得られる（位置を省略して `x[:]` とすれば全体のコピーが得られる）。

以上の方法によるコピーは、単に同じ要素の参照を持つ新しいオブジェクトである。このようなコピーを**浅いコピー**と呼ぶ。

In [ ]:
vals = ["a", "b", "c", "d"]
v_ref = vals.copy()  # オブジェクトのコピーが代入される
assert v_ref == vals
v_ref[1] = "e"
assert vals != v_ref == ["a", "e", "c", "d"]  # 変更が vals に及ばない

標準ライブラリの `copy` モジュールが提供する `copy()` 関数は、任意の型のインスタンスにも適用できる、浅いコピーを返す関数である。もし `__copy__()` メソッドが定義されていれば、`copy.copy()` 関数はそれを呼び出し、得られた結果を返す。

In [ ]:
import copy

class MyClass:
    def __init__(self, x):
        self.x = x

    def __copy__(self):
        print("Calling __copy__")
        return MyClass(self.x)

obj1 = MyClass(10)
obj2 = copy.copy(obj1)  # copy.copy を使ってコピー（__copy__が呼ばれる）
assert obj1.x == obj2.x == 10
obj2.x = 20
assert obj1.x != obj2.x  # 変更が obj1 に及ばない

Calling __copy__


### 深いコピー ###

浅いコピーでも、要素としてミュータブルなオブジェクトが含まれる場合、そのオブジェクトの変更が、オブジェクトを参照する全ての変数に影響する。

In [ ]:
vals = ["a", "b", ["c", "d"]]
v_ref = vals.copy()  # オブジェクトのコピーが代入される
v_ref[2][0] = "f"
print(f"{vals=},  {v_ref=}")  # 入れ子のリストの変更は vals にも及ぶ

vals=['a', 'b', ['f', 'd']],  v_ref=['a', 'b', ['f', 'd']]


`copy.deepcopy()` 関数は、新しいオブジェクトを作成し、その中の要素である子オブジェクトを再帰的にコピーする。つまり、ミュータブルな子オブジェクトについては、同じ値を持つ新しいオブジェクトを作成する。このようなコピーを**深いコピー**と呼ぶ。深いコピーでは、元のオブジェクトの変更による影響を受けない。

In [ ]:
import copy
vals = ["a", "b", ["c", "d"]]
v_ref = copy.deepcopy(vals)  # オブジェクトの深いコピーが代入される
assert v_ref == vals
v_ref[2][0] = "f"
print(f"{vals=},  {v_ref=}")  # 入れ子のリストの変更も vals に及ばない

vals=['a', 'b', ['c', 'd']],  v_ref=['a', 'b', ['f', 'd']]


深いコピーのデメリットは、オブジェクトの階層が深くなるほど再帰的なコピーに時間がかかることである。また、ミュータブルな子オブジェクトについて、すべて新たなオブジェクトが作成されるため、メモリ消費量が大きくなる。浅いコピーを選ぶか深いコピーを選ぶかは、用途を考慮して決める必要がある。

`copy.deepcopy()` 関数は、任意の型のインスタンスにも適用できる。もし `__deepcopy__()` メソッドが定義されていれば、`copy.deepcopy()` 関数はそれを呼び出し、得られた結果を返す。

`copy.copy()` と `copy.deepcopy()` 関数は、モジュール、メソッド、スタックトレース、スタックフレーム、ファイル、ソケット、ウィンドウ、その他これらに類似の型をコピーしない。


### copy.replace ###

Python 3.13 からは、`copy` モジュールに次の関数が追加された。

``` python
copy.replace(obj, /, **changes)
```

この関数は、第 1 引数 `obj` が持つ `__replace__()` メソッドを呼び出し、その結果を返す。第 2 引数以降のキーワード引数は `__replace__()` メソッドに渡される。

`__replace__()` は、オブジェクトの属性（の一部）を書き換えて、同じ型の新しいオブジェクトを作成するメソッドである。`replace()` メソッドによって呼び出される。

たとえば、`datetime.date`、`datetime.time`、`datetime.datetime` オブジェクトについては、`replace()` メソッドを使っても、`copy.replace()` 関数を使っても同じことができる。

``` python
import copy
import datetime

d1 = datetime.date(2024, 6, 30)
d2 = d1.replace(year=2025)
d3 = copy.replace(d1, year=2025)
assert d2 == d3
```

弱参照（weakref）
-----------------

参照カウントの働きに気づかないと、不要になったオブジェクトがプログラムのプロセス中ずっとメモリ上に存在し続けるということがある。たとえば、次のコードでは、`Player` クラスのインスタンスがリスト `players` や `card` によって参照されている。`name` 属性が `'武田'` のオブジェクトは両方に登録されるが、次の行で `players` から削除されている。これによりオブジェクト自体が削除されるわけではなく、依然として `card` から参照できることがわかる:

In [ ]:
class Player:
    def __init__(self, id, name):
        self.id = id
        self.name = name

    def __repr__(self):
        return f"Player({self.id}, {self.name})"

players = [Player(32, "武田"), Player(51, "上杉"), Player(66, "織田")]
card = [players[0], players[1]]
del players[0]
players, card  # players からは武田が除去されるが、card では依然として武田を参照できている

([Player(51, 上杉), Player(66, 織田)], [Player(32, 武田), Player(51, 上杉)])

このように、オブジェクトが複数の変数や他のオブジェクトによって参照されている場合、それらとオブジェクトのバインディングをすべて削除して参照カウントが 0 になるようにしないと、メモリ上にはオブジェクトが存在し続ける。

この問題を解決するため、標準ライブラリの `weakref` モジュールは、参照カウントを増やさないオブジェクトへの参照をサポートする。このような参照を**弱参照**と呼ぶ。弱参照と区別するため、普通の参照（参照カウントが増える参照）を**強参照**と呼ぶことがある。

ただし、全てのオブジェクトが弱参照で参照できるわけではない。以下のオブジェクトは弱参照をサポートする。

  * クラスインスタンス（`__slots__` 属性を持つクラスを除く）
  * （C ではなく） Python で書かれた関数
  * インスタンスメソッド
  * `set` オブジェクト
  * `frozenset` オブジェクト
  * ファイルオブジェクト
  * ジェネレーターオブジェクト
  * `socket` オブジェクト（`socket` モジュール）
  * `array` オジェクト（`array` モジュール）
  * `deque` オブジェクト（`deque` モジュール）
  * 正規表現パターンオブジェクト（`re` モジュール）

いくつかの組み込み型 `int`, `float`, `str`, `bytes`, `bytearray`, `list`, `tuple`, `dict` は弱参照を直接サポートしないことに注意する。`list` と `dict` に限っては、サブクラス化を行えば、そのサブクラスが弱参照をサポートする。

弱参照をサポートするオブジェクトを代入した変数 `x` に対して、`weakref.ref(x)` は、オブジェクトへの弱参照を返す。弱参照は、後ろに `()` を付けて呼び出すと、オブジェクトの参照を取得する。`del x` とした後は、弱参照は `None` を返す。

In [ ]:
import weakref
import sys

# dict のサブクラス化（弱参照のサポート）
class Dict(dict):
    pass

x = Dict(name="hoge")
assert sys.getrefcount(x) == 2
x_ref = weakref.ref(x)  # 弱参照
assert sys.getrefcount(x) == 2  # 参照カウントは増えない
assert x_ref()["name"] == "hoge"  # x_ref() でインスタンスを参照
del x
assert x_ref() is None  # del x のあとはインスタンスを参照できない

冒頭のコードで弱参照を使うと、次のようになる。

In [ ]:
import weakref

class Player:
    def __init__(self, id, name):
        self.id = id
        self.name = name

players = [Player(32, "武田"), Player(51, "上杉"), Player(66, "織田")]
card = [weakref.ref(players[0]), weakref.ref(players[1])]
assert card[0]().name == "武田"
del players[0]
assert card[0]() is None  # card からも参照できなくなる

`weakref` モジュールは、弱参照を扱うデータ型も提供している。

``` python
weakref.WeakKeyDictionary([dict])
```

キーを弱参照するマッピングクラスのインスタンスを生成する。キーへの強参照がなくなったときに、辞書のエントリは捨てられる。

``` python
weakref.WeakValueDictionary([dict])
```

値を弱参照するマッピングクラスのインスタンスを生成する。値への強参照が存在しなくなったときに、辞書のエントリは捨てられる。

In [ ]:
from weakref import WeakValueDictionary
import sys
import gc

class T:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"T({self.name})"

v1, v2 = T("spam"), T("ham")
assert sys.getrefcount(v1) == 2
assert sys.getrefcount(v2) == 2
d = dict()
wd = WeakValueDictionary()
d[1] = v1
wd[1] = v2
assert sys.getrefcount(v1) == 3  # 参照カウントが増えている
assert sys.getrefcount(v2) == 2  # 参照カウントが増えていない
assert len(d) == len(wd) == 1  # それぞれ1件のデータが格納されている
del v1, v2  # オブジェクトを削除
gc.collect()  # 手動でガベージコレクションを強制的に実行する
assert len(d) == 1  # d[1] が参照しているので T("spam") は削除されない
print(f"{d[1]=}")
assert len(wd) == 0  # T("ham") 自体が削除され、wd[1] のエントリも捨てられる

d[1]=T(spam)


`weakref.WeakKeyDictionary` 型と `weakref.WeakValueDictionary` 型の型アノテーションは、`dict` と同様に、添字表記（[]）を使ってキーの型と値の型を指定しなければならない。

In [ ]:
from weakref import WeakValueDictionary

class S:
    def __init__(self, name: str) -> None:
        self.name = name

def f(x: WeakValueDictionary[str, S]) -> str:
    return x["hoge"].name

defaultdict
-----------

`collections` が提供する `defaultdict` は、辞書を拡張し、存在しないキーを参照したときにデフォルト値を返すことができるデータ型である。コンストラクタは次のとおり。

``` python
collections.defaultdict(default_factory=None, /[, ...])
```

`default_factory` 引数には、引数なしの呼び出し可能オブジェクトを指定できる。`collections.defaultdict` インスタンスに対して存在しないキーを参照した場合は、 `default_factory` を引数なしで呼び出した時の戻り値がデフォルト値となる。もし `default_factory` 引数が `None`（デフォルト）なら、普通の `dict` オブジェクトと同様に `KeyError` 例外が発生する。

残りの引数は、キーワード引数も含め、`dict` のコンストラクタに与えられた場合と同様に扱われる。

`default_factory` 引数に `list` または `set`、`dict` を指定すると、それぞれ空のリスト、集合、辞書がデフォルト値となる。これを使用すると、`in` 演算子でキーの存在を確認したり、`setdefault()` メソッドを使う手順を省略して、簡単にデータをグループ化することができる。

次のコードは、`default_factory` 引数に `list` を指定した `collections.defaultdict` の使用例である。

In [ ]:
from collections import defaultdict

animals = [("チーター", "哺乳類"),
           ("ダチョウ", "鳥類"),
           ("コウモリ", "哺乳類"),
           ("ツバメ", "鳥類"),
           ("クジラ", "哺乳類"),
           ("ペンギン", "鳥類")]
dd = defaultdict(list)  # 存在しないキーを参照したときのデフォルト値を list() の値（空のリスト）とする
for animal, classis in animals:
    dd[classis].append(animal)  # dd[classis] はキーが存在しない場合は、空のリストを作成しそれに追加する
print(list(dd.items()))

[('哺乳類', ['チーター', 'コウモリ', 'クジラ']), ('鳥類', ['ダチョウ', 'ツバメ', 'ペンギン'])]


`default_factory` 引数にジェネレーターを利用する場合、ジェネレーター関数そのものを指定することはできない。ジェネレーター関数の呼び出しはジェネレーターオブジェクトを返すからである。この場合は、 `default_factory` 引数に、ジェネレーターオブジェクトから値を取りだすラムダ式（`next()` 関数を使う形式）を渡すようにすればよい。

次のコードは、値を `1`, `2`, `3`, ... と返すジェネレーターオブジェクト `counter` を作成しており、 `collections.defaultdict` のインスタンス化に際して `default_factory` 引数に `counter` から値を取りだすラムダ式を渡している。こうして生成された `collections.defaultdict` インスタンスは、全てのキーが連番のデフォルト値を持つようになる。

In [ ]:
from collections import defaultdict

def count():
    i = 0
    while True:
        i += 1
        yield i

counter = count()
dd = defaultdict(lambda: next(counter))
assert dd["apple"] == 1
assert dd["peach"] == 2
assert dd["pear"] == 3

`collections.defaultdict` 型の型アノテーションは、`dict` と同様に、添字表記（`[]`）を使ってキーの型と値の型を指定しなければならない。

In [ ]:
from collections import defaultdict

def func(dd: defaultdict[str, list]):
    return dd["hoge"]

dd = defaultdict(str)
assert func(dd) == ""  # 静的型チェッカーがエラーを表示する

OrderedDict
-----------

`collections` が提供する `OrderedDict` は、普通の辞書のようであるが、挿入順序に関係する追加の機能がある:

  * `popitem()` メソッドは引数を渡すことができ、 `popitem(False)` の場合、挿入順序の先頭のキーと値の対を消去し、その対を `tuple` として返す。 `popitem(True)` および引数を伴わない `popitem()` の場合は普通の辞書の `popitem()` と同じである。
  * `move_to_end(key, last=True)` メソッドを持ち、デフォルトでは既存の `key` を挿入順序の末尾に移動する。引数 `last` が `False` の場合、先頭に移動する。
  * オブジェクトの等価演算 `==` は挿入順序を考慮する。すなわち、`==` は同じキーと値の組を持つだけでなく、キーの順番も同じである場合にのみ真と評価される。

なお、普通の辞書でも以下のことができる。

  * `move_to_end(key)` に相当する操作は、`pop(key)` と代入で実現できる。
  * 順序を考慮した等価演算は、 `p == q and all(k1 == k2 for k1, k2 in zip(p, q))` で実現できる。

In [ ]:
from collections import OrderedDict
import copy

items = [("0", 0), ("1", 1), ("2", 2)]

od1 = OrderedDict(items)
print(f"{od1=}")
od2 = copy.deepcopy(od1)
assert od1 == od2
od2.move_to_end("0")
print(f"{od2=}")
assert od1 != od2

d1 = dict(items)
print(f"{d1=}")
d2 = copy.deepcopy(d1)
assert d1 == d2
d2["0"] = d2.pop("0")  # キーが 0 の項目を取り除いてから追加
print(f"{d2=}")  # キーが 0 の項目が末尾に追加されている
assert d1 == d2
assert not (d1 == d2 and all(k1 == k2 for k1, k2 in zip(d1, d2)))  # 順序を考慮した等価演算

od1=OrderedDict([('0', 0), ('1', 1), ('2', 2)])
od2=OrderedDict([('1', 1), ('2', 2), ('0', 0)])
d1={'0': 0, '1': 1, '2': 2}
d2={'1': 1, '2': 2, '0': 0}


`collections.OrderedDict` は、順序操作に関係する操作を普通の辞書よりも高速に行うことができるが、反面、メモリ使用効率、反復処理の速度、更新操作のパフォーマンスが普通の辞書より劣る。このため、普通の辞書より `collections.OrderedDict` が適しているといえるケースは、頻繁な並べ替え処理が必要という特殊なケースに限られる。

`collections.OrderedDict` 型の型アノテーションは、`dict` と同様に、添字表記（`[]`）を使ってキーの型と値の型を指定しなければならない。

In [ ]:
def func(od: OrderedDict[str, int], key: str) -> tuple[str, int]:
    od.move_to_end(key)
    return od.popitem()

Counter
-------

`collections` が提供する `Counter` は、次の 2 つの特徴を持つ多重集合のように辞書を拡張したデータ型である。 ※多重集合―同じ要素が複数回出現できて、その出現回数が重要である集合

  1. コンストラクタには、引数としてイテラブルオブジェクトを渡すことが可能で、その場合には、要素を辞書のキーとし、そのカウントを値に持つ。`dict` のコンストラクタと同様の引数を渡すこともできる。
  2. 存在しない要素に対して `KeyError` を送出する代わりに 0 を返す。ただし、カウントを 0 に設定しても、要素はカウンタから取り除かれない（完全に取り除くには、del を使うこと）。

``` python
collections.Counter([iterable-or-mapping])
```

`collections.Counter` は辞書を拡張しているので、辞書と同様に生成することもできるが、値は整数にする必要がある（例: `Counter(a=1, b=2)`）。値は負の数でもよい。

`collections.Counter` オブジェクトは、辞書で利用できるメソッドのうち、クラスメソッド `fromkeys()` を除いたすべてのメソッドをサポートしている。ただし、`update()` メソッドの振る舞いが辞書と異なる。また、追加のメソッドをサポートする。これらは以下のとおり:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `elements()` | それぞれの要素を、そのカウント分の回数だけ繰り返すイテレーターを返す。要素は挿入した順番で返される。<br />カウントが 1 未満の要素は無視される | イテレーター |
| `most_common([n])` | 要素とカウントのタプルを、カウントが多いものから少ないものまで順に並べたリストを返す。n に整数を指定<br />すると、最大 n 件の要素を返す。等しいカウントの要素は挿入順に並べられる | `list` |
| `total()` | カウントの合計を計算する | `int` |
| `subtract([iterable-or-mapping])` | 要素のカウントから `iterable` の要素のカウントまたは `mapping` の要素の値が引かれる。存在しない要素は<br /> 0 として処理される | `None` |
| `update([iterable-or-mapping])` | 要素のカウントに `iterable` の要素のカウントまたは `mapping` の要素の値が加算される。存在しない要素は<br /> 0 として処理される。`dict` の `update()` とは異なり、`iterable` には `(key, value)` 対のシーケンスではな<br />く、要素のシーケンスが求められる | `None` |

`collections.Counter` は `+`, `-` 演算子をサポートする。`c + d` はカウントの加算、`c - d` はカウントの減算であり、それぞれ `subtract()` メソッド、`update()` メソッドと異なり、`c` や `d` を変更せず新しい `collections.Counter` オブジェクトを返す。さらに、和集合演算子 `|` と積集合演算子 `&` もサポートされている。

In [ ]:
from collections import Counter

zen = "Beautiful is better than ugly"
cnt = Counter(zen)
assert cnt["e"] == zen.count("e") == 3
assert cnt["z"] == zen.count("z") == 0  # 存在しないキーを参照してもエラーにならない
assert "z" not in cnt  # キーに対する値(カウント) 0 が返される場合でもキーが存在するわけでない
print(list(cnt.items()))
cnt["z"] += 1  # 初期化しなくても +=1 でカウントできる
assert "z" in cnt
assert cnt["z"] == 1

c = Counter(spam=5, ham=2)
d = Counter(spam=1, egg=3)
assert list(c.elements()) == ["spam", "spam", "spam", "spam", "spam", "ham", "ham"]
assert (c + d).most_common(2) == [("spam", 6), ("egg", 3)]
assert list((c | d).items()) == [("spam", 5), ("ham", 2), ("egg", 3)]  # 和集合演算では最大値を返す
assert list((c & d).items()) == [("spam", 1)]  # 積集合演算では最小値を返す
c.subtract(d)
assert list(c.items()) == [("spam", 4), ("ham", 2), ("egg", -3)]
c.update(d)
assert list(c.items()) == [("spam", 5), ("ham", 2), ("egg", 0)]
assert c.total() == 7

[('B', 1), ('e', 3), ('a', 2), ('u', 3), ('t', 4), ('i', 2), ('f', 1), ('l', 2), (' ', 4), ('s', 1), ('b', 1), ('r', 1), ('h', 1), ('n', 1), ('g', 1), ('y', 1)]


`collections.Counter` 型の型アノテーションは、添字表記（`[]`）を使ってキーの型を指定しなければならない。値の型は指定しない（必ず `int` なので）。

In [ ]:
from collections import Counter

def func(c: Counter[str]):
    return c["hoge"]

c = Counter()
assert func(c) == 0

enum
----

### 列挙型 ###

標準ライブラリの `enum` モジュールが提供する `enum.Enum` のサブクラスは、列挙型と呼ばれ、複数の定数を列挙するデータ型である。

列挙型の定義は、`enum.Enum` を継承するクラスにクラス属性を定義する形となる。このクラス属性の名前は、列挙型の定数（**メンバー**と呼ばれる）の名前となる。クラスのメソッドや属性との名前の衝突の問題を回避するため、列挙型のメンバーの名前には UPPER_CASE の名前を使うことが推奨されている（公式ドキュメント [列挙型 HOWTO](https://docs.python.org/ja/3/howto/enum.html)）。同じ名前のメンバーを複数持つことはできない。

列挙型のメンバーを参照する方法は、次の 3 通りある:

  * 一般のクラス属性と同様にドット `.` で参照する。
  * 辞書のように定数の名前をキーとして `['CONSTANT_NAME']` で参照する。
  * クラス名の右に `(5)` のように付けて、定数値で参照する。

列挙型のメンバーの型は、そのメンバーの属する列挙型となる。列挙型は、`enum.Enum` から読み取り専用プロパティ `name` と `value` を継承する。それぞれ、列挙型のメンバーの名前と値を返す。

In [ ]:
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

assert isinstance(Color.RED, Color)
assert Color.RED.name == Color["RED"].name == Color(1).name == "RED"
assert Color.RED.value == Color["RED"].value == Color(1).value == 1
print(Color.RED)
Color.RED

Color.RED


<Color.RED: 1>

このように、列挙型のメンバーに対する `repr()` は、列挙型の名前、および、列挙型のメンバーの名前と値を表示する。列挙型のメンバーに対する `str()` は、列挙型の名前と列挙型のメンバーの名前のみを表示する。

同じ名前のメンバーを複数持つことはできないが、同じ値を持つメンバーを複数持つことはできる。値が重複するメンバーは、先に出現したメンバーに対するエイリアス（別名）となる。

In [ ]:
from enum import Enum

class Shape(Enum):
    SQUARE = 2
    DIAMOND = 1
    CIRCLE = 3
    ALIAS_FOR_SQUARE = 2

assert Shape.ALIAS_FOR_SQUARE.name == Shape.SQUARE.name == "SQUARE"

列挙型はイテラブルであり、反復処理の場面で、列挙型のメンバーを出現した順序にしたがって 1 つずつ取得できるイテレーターを返す。このイテレーターは、エイリアスとなる列挙型のメンバーを取得しない。

In [ ]:
assert [x.name for x in Shape] == ["SQUARE", "DIAMOND", "CIRCLE"]

クラスデコレーター `enum.unique()` を付けて列挙型を定義すると、同じ値のエイリアスとして複数の名前を設定できなくなり、メンバーの値の一意性が保証される。同じ値を設定すると、`ValueError` 例外が発生する。

``` python
from enum import Enum, unique

@unique
class Shape(Enum):
    SQUARE = 2
    DIAMOND = 1
    CIRCLE = 3
    ALIAS_FOR_SQUARE = 2

# エラーが発生して、次のようなメッセージが表示される
# Traceback (most recent call last):
#  (省略)
# ValueError: duplicate values found in <enum 'Shape'>: ALIAS_FOR_SQUARE -> SQUARE
```

列挙型のメンバーの値そのものは重要でない場合には、`enum.auto()` により値を自動的に設定することができる。

In [ ]:
from enum import Enum, auto

class Color(Enum):
    RED = auto()
    BLUE = auto()
    GREEN = auto()

Color.RED, Color.BLUE, Color.GREEN

(<Color.RED: 1>, <Color.BLUE: 2>, <Color.GREEN: 3>)

同じ列挙型のメンバーは、同一性を比較できる。等価の比較もできる。列挙型のメンバーの大小比較はサポートされていない（列挙型のメンバーは整数ではない）。

In [ ]:
assert Color.RED is Color.RED
assert Color.RED == Color.RED
assert Shape.SQUARE is Shape.ALIAS_FOR_SQUARE  # エイリアスは同一性を持つ
assert Shape.SQUARE == Shape.ALIAS_FOR_SQUARE
# Color.RED < Color.BLUE  # これはエラーが発生する

これに対して、**他の型（他の列挙型を含む）で定義された定数とは、値が同じでも等価とはならない**。

In [ ]:
WARNING = 1

class TrafficLight(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

assert Color.RED != WARNING
assert Color.RED != TrafficLight.RED

列挙型のメンバーを引数に取る関数を定義するとき、型アノテーションを付けると、静的型チェッカーにより実行前に間違った引数指定をチェックできるし、エディタの入力支援が得られる。

In [ ]:
def print_color(color: Color) -> None:
    print(color)

print_color(Color.RED)
# print_color("RED") のような間違った引数指定は静的型チェッカーで検出できる

Color.RED


Python 3.10 から導入された構造的パターンマッチングで、列挙型のメンバーをバリューパターンに使うことができる。

In [ ]:
def check(color: Color):
    match color:
        case Color.RED:
            print("赤")
        case Color.BLUE:
            print("青")
        case Color.GREEN:
            print("緑")
        case _:
            print("Color型ではない")

check(Color.GREEN)

緑


列挙型は、一般のクラスと同様にメソッドを追加することができる。

In [ ]:
from datetime import date
from enum import Enum

class Weekday(Enum):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 3
    THURSDAY = 4
    FRIDAY = 5
    SATURDAY = 6
    SUNDAY = 7

    @classmethod
    def from_date(cls, date: date):
        return cls(date.isoweekday())

assert Weekday.from_date(date(2024, 1, 1)) is Weekday.MONDAY

列挙型の値に整数を使う理由は、単に短くて便利というだけである。大半の使用例では、列挙型のメンバーの実際の値が何であるかは意識しない。しかし、値が重要な場合、列挙型は任意の値を持つことができる。

列挙型のサブクラス化は次のように制限される。

  1. 親の列挙型はメンバーが 1 つも定義されていないことが必要である。
  2. 1 つの列挙型のみ継承できる。多重継承は、親の列挙型 1 つのほかは、具象データ型を 1 つ、複数の `object` ベースの Mixin クラスだけが許容される。これらの基底クラスの順序は次の通り:

``` python
class EnumName([mix-in, ...,] [data-type,] base-enum):
    pass
```

In [ ]:
from enum import Enum

class GetVal:
    @classmethod
    def getval(cls, member_name: str):
        return cls[member_name].value

class Foo(Enum):
    def some_behavior(self):
        pass

class Bar(GetVal, Foo):
    HAPPY = 1
    SAD = 2

print(Bar.HAPPY)
assert Bar.getval("HAPPY") == 1

Bar.HAPPY


### 派生列挙型 ###

`enum.IntEnum` は `enum.Enum` の派生型であり、 `int` のサブクラスでもある。

  * `IntEnum` 列挙型のメンバーは整数と比較できるほか、異なる `IntEnum` 列挙型同士でも比較できる。もちろん、`enum.Enum` 列挙型のメンバーとは比較できない。
  * `IntEnum` 列挙型のメンバーは他の用途では整数のように振る舞う。

In [ ]:
from enum import IntEnum

class Shape(IntEnum):
    CIRCLE = 1
    SQUARE = 2

class Request(IntEnum):
    POST = 1
    GET = 2

assert Shape.CIRCLE == 1
assert Shape.CIRCLE == Request.POST
assert Shape.SQUARE > 1
assert Shape.CIRCLE * 10 + Shape.SQUARE == 12
assert ["a", "b", "c"][Shape.CIRCLE] == "b"
assert [i for i in range(Shape.SQUARE)] == [0, 1]

`enum.StrEnum` （Python 3.11 で追加）は `enum.Enum` の派生型であり、 `str` のサブクラスでもある。

  * `StrEnum` 列挙型のメンバーは文字列と比較できるほか、異なる `StrEnum` 列挙型同士でも比較できる。もちろん、`enum.Enum` 列挙型のメンバーとは比較できない。
  * `StrEnum` 列挙型のメンバーは他の用途では文字列のように振る舞う。
  * `StrEnum` 列挙型 で `enum.auto()` を使用すると、メンバー名を小文字に変換したものが値となる。

``` python
>>> from enum import StrEnum, auto
>>> class Direction(StrEnum):
...     EAST = auto()
...     WEST = auto()
...     NORTH = auto()
...     SOUTH = auto()
...
>>> Direction.EAST
<Direction.EAST: 'east'>
```

`enum.Flag` は `enum.Enum` の派生型であり、メンバーに対するビット演算をサポートする。メンバーに対するビット演算の結果も `enum.Flag` 列挙型のメンバーになる。 `enum.Flag` 列挙型のメンバーの値を 2 の累乗の整数とすると、ビット演算が論理演算としての意味を持つようになる。`enum.Flag` 列挙型で `enum.auto()` を使用すると、 1 から始まる 2 の累乗の整数になる。

In [ ]:
from enum import Flag, auto

class Weekday(Flag):
    MONDAY = auto()  # 1
    TUESDAY = auto()  # 2
    WEDNESDAY = auto()  # 4
    THURSDAY = auto()  # 8
    FRIDAY = auto()  # 16
    SATURDAY = auto()  # 32
    SUNDAY = auto()  # 64
    WEEKEND = SATURDAY | SUNDAY

assert Weekday.WEEKEND.value == 96
print(repr(Weekday.MONDAY | Weekday.WEDNESDAY | Weekday.FRIDAY))  # 月水金
for day in Weekday:
    if day & (Weekday.MONDAY | Weekday.WEDNESDAY | Weekday.FRIDAY):
        print(day.name)

<Weekday.FRIDAY|WEDNESDAY|MONDAY: 21>
MONDAY
WEDNESDAY
FRIDAY


`enum.IntFlag` は `enum.Enum` の派生型であり、 `enum.Flag` と似ているが、 `int` のサブクラスでもある。このため、 `enum.IntEnum` と同様の機能をサポートする。

データの整形表示
----------------

### pprint ###

標準ライブラリの `pprint` モジュールが提供する `pprint.pprint()` 関数は、何重にもネストした辞書や、要素数が長いリストのように、人間が読みにくい構造のデータを整形して出力する。

``` python
pprint.pprint(object, stream=None, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True, underscore_numbers=False)
```

| 引数 | 意味 |
|:--|:--|
| `object` | 出力するデータを指定する |
| `stream` | 出力先のファイルオブジェクトを指定する。`None`（デフォルト値）ならば、標準出力 `sys.stdout` に出力される |
| `indent` | ネストしたオブジェクトの子要素を出力するときのインデント数を指定する |
| `width` | 出力幅を指定する |
| `depth` | ネストしたオブジェクトを出力する際の、最大レベル数を指定する。`None`（デフォルト値）ならば、すべてのレベルを出力する |
| `compact` | キーワード専用引数。`True` の場合、各行に `width` の幅に収まるだけの要素が出力される。`False` の場合、1 行ごとに 1 要素が出力される |
| `sort_dicts` | キーワード専用引数。`True` の場合、キーがソートされた状態で出力される。`False` の場合、挿入順に出力される |
| `underscore_numbers` | キーワード専用引数。`True` の場合、整数は千の位の区切り文字としてアンダースコア `_` が使用される。`False` の場合、アンダースコアは使<br />用されない |

In [ ]:
import pprint
stuff = ["spam", "eggs", "lumberjack", "knights", "ni"]
stuff.insert(0, stuff)
pprint.pprint(stuff)

[<Recursion on list with id=137559039340736>,
 'spam',
 'eggs',
 'lumberjack',
 'knights',
 'ni']


``` python
pprint.pformat(object, indent=1, width=80, depth=None, *, compact=False, sort_dicts=True, underscore_numbers=False)
```

この関数は、`pprint.pprint()` 関数と同じくオブジェクトを整形するが、標準出力に印字するのではなく、戻り値として文字列を返す。

### tabulate ###

サードパーティ製の [tabulate](https://pypi.org/project/tabulate/) は、ネストされたリストなど一定のデータ構造を表形式に整形して出力する関数を提供する。ライセンスは MIT License。インストールは次のとおり。

``` shell
pip install tabulate
```

``` python
tabulate(tabular_data, headers=(), tablefmt="simple", floatfmt=_DEFAULT_FLOATFMT, intfmt=_DEFAULT_INTFMT, numalign=_DEFAULT_ALIGN,
         stralign=_DEFAULT_ALIGN, missingval=_DEFAULT_MISSINGVAL, showindex="default", disable_numparse=False, colglobalalign=None,
         colalign=None, preserve_whitespace=False, maxcolwidths=None, headersglobalalign=None, headersalign=None, rowalign=None,
         maxheadercolwidths=None, break_long_words=_BREAK_LONG_WORDS, break_on_hyphens=_BREAK_ON_HYPHENS)
```

この関数は、一定のデータ構造を表形式に整形したテキストを生成して返す。以下のデータ構造をサポートする。

| データ構造 | 例 |
|:---|:---|
| ネストされたリスト | `[["Name", "Age"], ["Alice", 24], ["Bob", 19]]` |
| リストを含む辞書 | `{"Name": ["Alice", "Bob"], "Age": [24, 19]}` |
| 辞書のリスト | `[{"Name": "Alice", "Age": 24}, {"Name": "Bob", "Age": 19}]` |

関数の主な引数は、以下の通り。

| 引数 | 意味 |
|:---|:---|
| `tabular_data` | 入力データ |
| `headers` | 表のヘッダーを指定する。<br /><br />・文字列のリスト: 表のヘッダーを直接指定する<br /><br />・`'firstrow'`: `tabular_data` がネストされたリストの場合、最初の内側のリストを表のヘッダーとして扱う<br /><br />・`'keys'`: `tabular_data` がリストを含む辞書や、辞書のリストである場合、各キーを表のヘッダーとする |
| `tablefmt` | 表の書式を文字列で指定する。デフォルトは `'simple'` |
| `floatfmt` | `float` 型データに対する書式指定。デフォルトは `'g'` |
| `intfmt` | `int` 型データに対する書式指定。デフォルトは 10 進数 |
| `numalign` | 数値の列の配置。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え）, `'decimal'` |
| `stralign` | 文字列の列の配置。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え） |
| `showindex` | `True` の場合、自動連番でインデックスを表示する。イテラブルの場合、表示する任意のインデックスを指定する |
| `colalign` | シーケンスで列ごとの配置を指定する。`'right'`（右揃え）, `'center'`（中央揃え）, `'left'`（左揃え） |

書式 `tablefmt='simple'` の例:

In [ ]:
# ネストされたリストを整形出力
from tabulate import tabulate
table = [["Sun",696000,1989100000], ["Earth",6371,5973.6], ["Moon",1737,73.5], ["Mars",3390,641.85]]
print(tabulate(table, headers=["Planet","R (km)", "mass (x 10^29 kg)"]))

Planet      R (km)    mass (x 10^29 kg)
--------  --------  -------------------
Sun         696000           1.9891e+09
Earth         6371        5973.6
Moon          1737          73.5
Mars          3390         641.85


書式 `tablefmt='plain'` の例:

In [ ]:
# ネストされたリストを整形出力
from tabulate import tabulate
print(tabulate([["Name", "Age"], ["Alice", 24], ["Bob", 19]],
               headers='firstrow', tablefmt='plain'))

Name      Age
Alice      24
Bob        19


書式 `tablefmt='grid'` の例:

In [3]:
# リストを含む辞書を整形出力
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19]},
               headers="keys", tablefmt='grid'))

+--------+-------+
| Name   |   Age |
+========+=======+
| Alice  |    24 |
+--------+-------+
| Bob    |    19 |
+--------+-------+


書式 `tablefmt='psql'` の例:

In [6]:
# 辞書のリストを整形出力
from tabulate import tabulate
print(tabulate([{"Name": "Alice", "Age": 24}, {"Name": "Bob", "Age": 19}],
               headers="keys", tablefmt='psql'))

+--------+-------+
| Name   |   Age |
|--------+-------|
| Alice  |    24 |
| Bob    |    19 |
+--------+-------+


書式 `tablefmt='pretty'` の例:

In [9]:
# インデックスを表示
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19]},
               headers="keys", tablefmt='pretty', showindex=True))

+---+-------+-----+
|   | Name  | Age |
+---+-------+-----+
| 0 | Alice | 24  |
| 1 |  Bob  | 19  |
+---+-------+-----+


書式 `tablefmt='pipe'` の例:

In [8]:
# 列ごとの配置を指定
from tabulate import tabulate
print(tabulate({"Name": ["Alice", "Bob"], "Age": [24, 19], "Gender": ["female", "male"]},
               headers="keys", tablefmt='pipe', colalign=["left", "right", "center"]))

| Name   |   Age |  Gender  |
|:-------|------:|:--------:|
| Alice  |    24 |  female  |
| Bob    |    19 |   male   |


書式 `tablefmt='html'` の例:

In [16]:
# float 型データに対する書式指定
import math
from tabulate import tabulate
from IPython.display import HTML
display(HTML(tabulate({"constant": ["e", "pi"], "value": [math.e, math.pi]},
             headers="keys", tablefmt='html', floatfmt='.2f')))

constant,value
e,2.72
pi,3.14
